In [17]:

import numpy as np
import pandas as pd
fixedItemSet=np.load(r"hetrec2011-delicious-2k\frequentItemVec.npy")
userflow=np.load(r'hetrec2011-delicious-2k\userflow.npy')
userclick=np.load(r'hetrec2011-delicious-2k\userclick.npy',allow_pickle=True)

def hwithoutconstraints(action):
    if type(action)==torch.Tensor:
        action=action.detach().numpy()
    items=np.argwhere(np.array(action)>0.5).reshape(args.card)
    if t<args.card:
        itemlist=userclick[0][:args.card]
    else:
        itemlist=userclick[0][(t-args.card):min((t+args.card),len(userclick[0]))]
    cnt2=0
    for i in items:
        cnt2+=( itemlist.count(i))
    return cnt2/(args.card)

import argparse
from copy import deepcopy
import torch
import gym
import time
import torch
import torch.nn as nn	
import torch.nn.functional as F	
import torch.optim as optim	
import numpy as np

import argparse
import sys

import copy
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'/root/gurobi903/linux64/lib/python3.6_utf32/gurobipy')

class para:
    def __init__(self):
        self.batch_size=16
        self.wmc=10
        self.solDim=40
        self.card=10
        self.test_batch_size=1000
        self.epochs=10
        self.lr=0.1
        self.momentum=0.5
        self.no_cuda='store_true'
        self.seed=1
        self.log_interval=10
        self.hidden_dim=10
args=para()
thre=1
legalList={}
for i in range(40):
    legalList[i]=[]
for i in range(40):
    for j in range(40):
        if i!=j and np.linalg.norm(fixedItemSet[i]-fixedItemSet[j])<thre:
            legalList[i].append(j)

In [18]:

import argparse
from copy import deepcopy
import torch
import gym
import time
import torch
import torch.nn as nn	
import torch.nn.functional as F	
import torch.optim as optim	
import numpy as np

import argparse
import sys

import copy
import numpy as np
from numpy.random import permutation
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,'/root/gurobi903/linux64/lib/python3.6_utf32/gurobipy')


def checkFea(x):
    cnt=0
    cntT=0
    for i in range(len(legalList.keys())):
        for j in legalList[i]:
            cntT+=1
            if x[j]+x[i]>1:
                cnt+=1
    return cnt/cntT/2+abs(sum(x)-args.card)/args.card/2
def CB(alpha,x,M):
    return alpha*np.sqrt(np.dot(np.dot(x.T,np.linalg.inv(M)),x))
def solver(theta,MODEL):

    import gurobipy
    import time
    s=time.time()
    
    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    
    MODEL.update()

    
    MODEL.setObjective(np.array(variables).dot(theta), sense=gurobipy.GRB.MAXIMIZE)

    
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    
    MODEL.optimize()
    return np.array(MODEL.x)
def solver_quad(Q,MODEL):

    import gurobipy
    import time
    s=time.time()
    
    variables=[]
    for i in range(args.solDim):
        variables.append(MODEL.addVar(vtype=gurobipy.GRB.BINARY, name='x'+'i'))
    
    MODEL.update()

    
    MODEL.setObjective(np.array(variables).dot(Q).dot(np.array(variables)), sense=gurobipy.GRB.MAXIMIZE)

    
    cnt=0
    for i in range(args.solDim):
        if len(legalList)>0:
            for j in legalList[i]:
                MODEL.addConstr(-variables[i]-variables[j] >= -1, name=str(cnt))
                cnt+=1
    MODEL.addConstr(sum(variables) == args.card, name=str(cnt))
    f=time.time()
    
    
    print('??')
    MODEL.optimize()
    print('??')
    return np.array(MODEL.x)
class myenv2(gym.Env):
    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second': 50
    }

    def __init__(self):

        
        

        self.steps = torch.tensor(np.array([0.1]*args.solDim))
        self._max_episode_steps = 500
        self.iteration=1
        self.constraints=[]
        self.constraint=0
        self.history=np.ones(args.solDim)
    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        self.iteration+=1 
        done=1
        action=[action[i]+0.0*np.sqrt(np.log(self.iteration)/self.history[i]) for i in range(args.solDim)]
        action=np.array([1 if i in np.argsort(action)[-args.card:] else 0  for i in range(args.solDim)])

        
  
   
    
        
        
        reward=hwithoutconstraints(action)-10*checkFea(action)
        rewards.append(hwithoutconstraints(action))
        constraints.append(checkFea(action))
        self.history+=np.array(action)
        print(self.iteration,1*hwithoutconstraints(action),checkFea(action))
        
        return action, reward, done, {}

    def reset(self):
        return  np.array(np.array([0.1]*args.solDim)),

    def render(self, mode='human'):
        pass


In [19]:
import sys
import math

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.utils as utils
import torchvision.transforms as T
from torch.autograd import Variable
import pdb

class Policy(nn.Module):
    def __init__(self, hidden_size, num_inputs, action_space):
        super(Policy, self).__init__()
        self.action_space = action_space
        num_outputs = action_space

        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, num_outputs)

    def forward(self, inputs):
        x = inputs
        x = F.relu(self.linear1(x))
        action_scores = self.linear2(x)
        return action_scores#F.softmax(action_scores)

def top_k(x, k):
    return np.argpartition(x, k)[..., -k:]

def sample_k(logits, k):
    u = np.random.uniform(size=np.shape(logits))
    z = -np.log(-np.log(u))
    return top_k(logits +z, k)

class REINFORCE:
    def __init__(self, hidden_size, num_inputs, action_space):
        self.action_space = action_space
        self.model = Policy(hidden_size, num_inputs, action_space)
        self.optimizer = optim.Adam(self.model.parameters(), lr=1e-3)
        self.model.train()

    def select_action(self, state):
        logits=self.model(Variable(state))[0][0]
        u = torch.Tensor(state.shape).uniform_(0,1)
        z = -torch.log(-torch.log(u))
        rank=torch.argsort(-(logits+z))[0][0]
        action=(rank<args.card)+0.0
        probs=[]
        for kk in range(10):
            topK=list(range(args.card))
            np.random.shuffle(topK)
            cnt=0
            for i,j in enumerate(action):
                if j>0.5:
                    rank[i]=topK[cnt]
                    cnt+=1
            rankdict={}
            for i,j in zip(rank,logits):
                rankdict[i.item()]=torch.exp(j)
            prob=1.
            for j in range(args.card):
                prob*=(j+1)*(rankdict[j]/sum([rankdict[k] for k in list(range(j,args.card))]))
            probs.append(prob.view(1))
        probs=torch.cat([i for i in probs],0).mean()
        return action, torch.log(probs)

    def update_parameters(self, rewards0, log_probs, gamma):
        R = torch.zeros(1)
        loss = 0
        for i in reversed(range(len(rewards0))):
            R = gamma * R + rewards0[i]
            loss = loss - (log_probs[i]*(Variable(R).expand_as(torch.tensor([log_probs[i]])))).sum() 
        loss = loss / len(rewards0)
        
        self.optimizer.zero_grad()
        loss.backward()
        utils.clip_grad_norm(self.model.parameters(), 40)
        self.optimizer.step()
import gym


class NormalizedActions(gym.ActionWrapper):

    def _action(self, action):
        action = (action + 1) / 2  # [-1, 1] => [0, 1]
        action *= (self.action_space.high - self.action_space.low)
        action += self.action_space.low
        return action

    def _reverse_action(self, action):
        action -= self.action_space.low
        action /= (self.action_space.high - self.action_space.low)
        action = action * 2 - 1
        return actions


In [ ]:
rewards=[]
constraints=[]
import argparse, math, os
import numpy as np
import gym
from gym import wrappers

import torch
from torch.autograd import Variable
import torch.nn.utils as utils
class parser:
    def __init__(self):
        self.gamma=0.99
        self.exploration_end=100
        self.seed=123
        self.num_steps=1000
        self.num_episodes=2000
        self.hidden_size=128
        self.render='store_true'
        self.ckpt_freq=100
        self.display=False
args2 = parser()

env = myenv2()

torch.manual_seed(args2.seed)
np.random.seed(args2.seed)

agent = REINFORCE(args2.hidden_size, args.solDim,args.solDim)

dir = 'ckpt'
if not os.path.exists(dir):    
    os.mkdir(dir)
    

for i_episode in range(5000):
    state = torch.Tensor([env.reset()])
    entropies = []
    log_probs = []
    rewards0 = []
    for t in range(args2.num_steps):
        action, log_prob = agent.select_action(state)
        next_state, reward, done, _ = env.step(action)
        log_probs.append(log_prob)
        rewards0.append(reward)
        state = torch.Tensor([next_state])

        if done:
            break

    agent.update_parameters(rewards0, log_probs, args2.gamma)


    if i_episode%args2.ckpt_freq == 0:
        torch.save(agent.model.state_dict(), os.path.join(dir, 'reinforce-'+str(i_episode)+'.pkl'))
    if i_episode%1000==0:
        np.save('reward_gumbel_top_K_reinforce_realorderdel.npy',rewards)
        np.save('constraint_gumbel_top_K_reinforce_realorderdel.npy',constraints)

In [31]:
import numpy as np
with open('1.txt','r') as f:
    tmp=f.readlines()
r=[]
c=[]
for j in range(len(tmp)):
    i=tmp[j].strip().split(' ')
    r.append(float(i[1]))
    c.append(float(i[2]))

In [24]:
r

[22.18498603962269,
 21.429845199153267,
 22.120748573675513,
 20.644113392305627,
 22.05222336635929,
 23.52755370133691,
 22.19222708899271,
 21.122759027448655,
 19.530038728143435,
 19.10185934276589,
 20.7101770372958,
 21.82239286367965,
 22.229858913117404,
 21.772986097110554,
 23.177542655264787,
 22.747772997598645,
 20.914826822794574,
 19.99695866607256,
 23.71584684692924,
 21.89194936870176,
 21.845416979205467,
 22.685487616066666,
 23.493861854586896,
 21.036615868740306,
 23.242916596227516,
 22.17642472151365,
 21.106382617198616,
 21.76553344093229,
 20.72623338805634,
 21.34367231917765,
 20.97693588273044,
 18.557498233283752,
 20.11291943988537,
 21.400758447941275,
 20.55428971316345,
 20.802743420144406,
 19.901910536803246,
 22.775434754390997,
 22.863762725870043,
 24.172124917635635,
 21.48351067222693,
 23.29513860413174,
 19.529003118286123,
 21.290870306782,
 22.469346911075334,
 21.397844214445083,
 20.98630144952469,
 23.79163466170872,
 22.3677474450638

In [5]:
hwithoutconstraints = lambda x:10*((x.dot(a))**1)  #100*(x.dot(a))**1+100*x.dot(Q).dot(x) # 、100*x.dot(Q).dot(x)
rewards=[]
constraints=[]
import argparse, math, os
import numpy as np
import gym
from gym import wrappers

import torch
from torch.autograd import Variable
import torch.nn.utils as utils
class parser:
    def __init__(self):
        self.gamma=0.99
        self.exploration_end=100
        self.seed=123
        self.num_steps=1000
        self.num_episodes=2000
        self.hidden_size=128
        self.render='store_true'
        self.ckpt_freq=100
        self.display=False
args2 = parser()

env = myenv2()

torch.manual_seed(args2.seed)
np.random.seed(args2.seed)

agent = REINFORCE(args2.hidden_size, args.solDim,args.solDim)

dir = 'ckpt'
if not os.path.exists(dir):    
    os.mkdir(dir)
    

for i_episode in range(5000):
    state = torch.Tensor([env.reset()])
    entropies = []
    log_probs = []
    rewards0 = []
    for t in range(args2.num_steps):
        action, log_prob = agent.select_action(state)
        next_state, reward, done, _ = env.step(action)
        log_probs.append(log_prob)
        rewards0.append(reward)
        state = torch.Tensor([next_state])

        if done:
            break

    agent.update_parameters(rewards0, log_probs, args2.gamma)


    if i_episode%args2.ckpt_freq == 0:
        torch.save(agent.model.state_dict(), os.path.join(dir, 'reinforce-'+str(i_episode)+'.pkl'))
    if i_episode%1000==0:
        np.save('reward_gumbel_top_K_reinforce_1order300.npy',rewards)
        np.save('constraint_gumbel_top_K_reinforce_1order300.npy',constraints)

NameError: name 'spaces' is not defined

In [27]:
len(rewards)

5000

In [192]:
hwithoutconstraints = lambda x:10*((x.dot(a))**3)  #100*(x.dot(a))**1+100*x.dot(Q).dot(x) # 、100*x.dot(Q).dot(x)
rewards=[]
constraints=[]
import argparse, math, os
import numpy as np
import gym
from gym import wrappers

import torch
from torch.autograd import Variable
import torch.nn.utils as utils
class parser:
    def __init__(self):
        self.gamma=0.99
        self.exploration_end=100
        self.seed=123
        self.num_steps=1000
        self.num_episodes=2000
        self.hidden_size=128
        self.render='store_true'
        self.ckpt_freq=100
        self.display=False
args2 = parser()

env = myenv2()

torch.manual_seed(args2.seed)
np.random.seed(args2.seed)

agent = REINFORCE(args2.hidden_size, args.solDim,args.solDim)

dir = 'ckpt'
if not os.path.exists(dir):    
    os.mkdir(dir)
    

for i_episode in range(5000):
    state = torch.Tensor([env.reset()])
    entropies = []
    log_probs = []
    rewards0 = []
    for t in range(args2.num_steps):
        action, log_prob = agent.select_action(state)
        next_state, reward, done, _ = env.step(action)
        log_probs.append(log_prob)
        rewards0.append(reward)
        state = torch.Tensor([next_state])

        if done:
            break

    agent.update_parameters(rewards0, log_probs, args2.gamma)


    if i_episode%args2.ckpt_freq == 0:
        torch.save(agent.model.state_dict(), os.path.join(dir, 'reinforce-'+str(i_episode)+'.pkl'))
    if i_episode%1000==0:
        np.save('reward_gumbel_top_K_reinforce_3order100.npy',rewards)
        np.save('constraint_gumbel_top_K_reinforce_3order100.npy',constraints)

2 86.19698909234715 0.019178082191780823


<ipython-input-186-ff246c2564bb>:80: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  utils.clip_grad_norm(self.model.parameters(), 40)


3 62.87971564667858 0.010958904109589041
4 55.67178724133857 0.021917808219178082
5 53.30842094287104 0.00821917808219178
6 65.84757149763033 0.009589041095890411
7 78.70395026072684 0.021917808219178082
8 77.59274451450307 0.019178082191780823
9 65.0005429201153 0.021917808219178082
10 82.622871930539 0.023287671232876714
11 85.8178383287229 0.03424657534246575
12 109.48170712334621 0.06575342465753424
13 78.85561427737673 0.036986301369863014
14 83.36350455029489 0.028767123287671233
15 101.5383097049436 0.021917808219178082
16 57.805199380536195 0.015068493150684932
17 76.29732892694422 0.021917808219178082
18 81.31963170535647 0.024657534246575342
19 55.62603944415834 0.010958904109589041
20 63.95486147370373 0.01780821917808219
21 63.15825177797686 0.024657534246575342
22 64.85466523226115 0.0136986301369863
23 65.73467659011368 0.02054794520547945
24 79.08112629676897 0.026027397260273973
25 48.39716945679023 0.00684931506849315
26 53.92202374904501 0.010958904109589041
27 78.087

199 62.32057060148765 0.010958904109589041
200 62.096764476917706 0.0136986301369863
201 60.189279127819084 0.010958904109589041
202 56.21179689046752 0.00684931506849315
203 70.56149987306087 0.026027397260273973
204 65.86333358086857 0.005479452054794521
205 61.65679226335968 0.010958904109589041
206 67.37644289839741 0.019178082191780823
207 72.75820258759965 0.01780821917808219
208 58.26783691703589 0.01643835616438356
209 84.18413419319663 0.028767123287671233
210 84.84339578040674 0.015068493150684932
211 79.07839505113193 0.04246575342465753
212 92.71033357179157 0.038356164383561646
213 72.94860179300953 0.021917808219178082
214 115.7671763521492 0.03424657534246575
215 99.88046846949877 0.03561643835616438
216 60.55483332032366 0.010958904109589041
217 68.11246378281264 0.010958904109589041
218 76.01900565060825 0.012328767123287671
219 67.39985267457934 0.019178082191780823
220 63.250298365036876 0.043835616438356165
221 74.57697981245283 0.023287671232876714
222 99.186629764

392 83.65529993349452 0.026027397260273973
393 84.56054834731063 0.021917808219178082
394 58.7322838545791 0.012328767123287671
395 62.67848543006728 0.015068493150684932
396 72.46838494510098 0.00821917808219178
397 52.809274007809954 0.028767123287671233
398 91.33985106814556 0.038356164383561646
399 81.75217148671614 0.02054794520547945
400 55.183633673254846 0.005479452054794521
401 54.38899025800374 0.00410958904109589
402 85.95882438275598 0.03972602739726028
403 70.95128011538233 0.01780821917808219
404 70.96890673304848 0.026027397260273973
405 77.45747892181804 0.03287671232876712
406 89.60074496937335 0.023287671232876714
407 56.412884153037595 0.019178082191780823
408 51.61512733939503 0.01780821917808219
409 109.133218931547 0.01780821917808219
410 77.48856745504602 0.012328767123287671
411 79.95818181531058 0.01643835616438356
412 75.18443094690525 0.02054794520547945
413 46.136437000795596 0.00821917808219178
414 73.12849416725622 0.028767123287671233
415 90.0814287785032

...

4185 74.03896366371688 0.031506849315068496
4186 64.11927934614428 0.019178082191780823
4187 50.28989668867793 0.01643835616438356
4188 79.0086567048216 0.015068493150684932
4189 77.57215187850267 0.028767123287671233
4190 73.9331042839061 0.00410958904109589
4191 72.76597005594282 0.01780821917808219
4192 52.46417643080683 0.021917808219178082
4193 51.89653581856958 0.01643835616438356
4194 73.82256031229743 0.0136986301369863
4195 63.95089291652276 0.00821917808219178
4196 61.968323288523415 0.009589041095890411
4197 63.02001109729193 0.00684931506849315
4198 56.23772632656944 0.005479452054794521
4199 91.66352294682538 0.010958904109589041
4200 71.76002225684476 0.012328767123287671
4201 47.31904114719373 0.009589041095890411
4202 95.24278140955289 0.021917808219178082
4203 68.1450442712396 0.012328767123287671
4204 103.1412809267868 0.02054794520547945
4205 63.618637510695876 0.03287671232876712
4206 69.95649642490794 0.010958904109589041
4207 46.77264906874168 0.00821917808219178


4374 70.1216714659855 0.0273972602739726
4375 72.40572225634 0.031506849315068496
4376 72.29019852051714 0.03287671232876712
4377 62.075864146983776 0.00684931506849315
4378 69.51330817160152 0.005479452054794521
4379 69.40452303684128 0.0136986301369863
4380 62.49740512343341 0.00821917808219178
4381 52.73146276314493 0.010958904109589041
4382 70.7205151748084 0.019178082191780823
4383 53.6368655707267 0.01643835616438356
4384 63.80839599898667 0.01780821917808219
4385 45.24678999052945 0.010958904109589041
4386 70.90154976773346 0.01780821917808219
4387 81.26085017055472 0.021917808219178082
4388 85.58563427469235 0.0136986301369863
4389 61.12088075239737 0.01780821917808219
4390 66.50095589143774 0.015068493150684932
4391 80.98617636778896 0.03287671232876712
4392 106.60311645613297 0.031506849315068496
4393 65.00573873657771 0.012328767123287671
4394 75.04762337175853 0.026027397260273973
4395 66.23953178348363 0.036986301369863014
4396 64.25412130729853 0.045205479452054796
4397 7

4562 74.66239026239221 0.01780821917808219
4563 85.39917553906947 0.01643835616438356
4564 61.758838786969406 0.030136986301369864
4565 64.02006630199449 0.019178082191780823
4566 73.82248150128595 0.012328767123287671
4567 52.39036345473907 0.0136986301369863
4568 76.18570275165993 0.023287671232876714
4569 52.52589602671027 0.024657534246575342
4570 73.24464307587101 0.023287671232876714
4571 88.15583631483958 0.03972602739726028
4572 76.26950916621364 0.0273972602739726
4573 74.41144050000123 0.031506849315068496
4574 75.21913342657035 0.01780821917808219
4575 107.81585781705027 0.031506849315068496
4576 61.87148685735468 0.02054794520547945
4577 51.21361828567135 0.015068493150684932
4578 78.47075582617916 0.01643835616438356
4579 73.35787355278237 0.015068493150684932
4580 82.95250733065764 0.03561643835616438
4581 55.49395931317697 0.00821917808219178
4582 62.38589734059744 0.00410958904109589
4583 47.6786505452018 0.023287671232876714
4584 87.80576431036675 0.015068493150684932


4751 82.30776608900028 0.030136986301369864
4752 102.95160671700863 0.01643835616438356
4753 101.63885666671158 0.021917808219178082
4754 79.26800136248575 0.009589041095890411
4755 43.696002735039116 0.015068493150684932
4756 68.03374142031787 0.026027397260273973
4757 78.89275495142351 0.019178082191780823
4758 74.81675011174012 0.021917808219178082
4759 79.13361595467927 0.024657534246575342
4760 68.47512908764872 0.012328767123287671
4761 76.98642253317729 0.012328767123287671
4762 67.80836983436689 0.009589041095890411
4763 79.86407698518991 0.01780821917808219
4764 69.8390175950938 0.01643835616438356
4765 79.59187615548196 0.030136986301369864
4766 54.80767838495013 0.0273972602739726
4767 75.43288846220246 0.0136986301369863
4768 69.1653356806264 0.005479452054794521
4769 51.84118493968603 0.03561643835616438
4770 42.91631717294141 0.009589041095890411
4771 102.28478275620871 0.026027397260273973
4772 65.3374008473516 0.019178082191780823
4773 95.94230046153676 0.01369863013698

4940 67.169332392871 0.0273972602739726
4941 92.66030130388907 0.021917808219178082
4942 58.57160977510492 0.015068493150684932
4943 68.80166842215426 0.0136986301369863
4944 58.385750118521415 0.024657534246575342
4945 82.4971482155574 0.02054794520547945
4946 93.75700223641154 0.01643835616438356
4947 61.409722233722746 0.010958904109589041
4948 56.48335424872137 0.026027397260273973
4949 56.51871093690755 0.010958904109589041
4950 90.0571753983374 0.01780821917808219
4951 77.7055647259208 0.010958904109589041
4952 81.07722775782811 0.01643835616438356
4953 57.941312249193814 0.02054794520547945
4954 64.3656927225847 0.012328767123287671
4955 69.72385047339229 0.0273972602739726
4956 93.40849188853568 0.012328767123287671
4957 41.29120380369467 0.009589041095890411
4958 74.56076347243813 0.023287671232876714
4959 69.2628144001705 0.010958904109589041
4960 71.46869534874693 0.015068493150684932
4961 84.72345291012034 0.01780821917808219
4962 76.70901912642663 0.01643835616438356
4963 

In [193]:
hwithoutconstraints = lambda x:10*((x.dot(a))**2+x.dot(Q).dot(x))  #100*(x.dot(a))**1+100*x.dot(Q).dot(x) # 、100*x.dot(Q).dot(x)
rewards=[]
constraints=[]
import argparse, math, os
import numpy as np
import gym
from gym import wrappers

import torch
from torch.autograd import Variable
import torch.nn.utils as utils
class parser:
    def __init__(self):
        self.gamma=0.99
        self.exploration_end=100
        self.seed=123
        self.num_steps=1000
        self.num_episodes=2000
        self.hidden_size=128
        self.render='store_true'
        self.ckpt_freq=100
        self.display=False
args2 = parser()

env = myenv2()

torch.manual_seed(args2.seed)
np.random.seed(args2.seed)

agent = REINFORCE(args2.hidden_size, args.solDim,args.solDim)

dir = 'ckpt'
if not os.path.exists(dir):    
    os.mkdir(dir)
    

for i_episode in range(5000):
    state = torch.Tensor([env.reset()])
    entropies = []
    log_probs = []
    rewards0 = []
    for t in range(args2.num_steps):
        action, log_prob = agent.select_action(state)
        next_state, reward, done, _ = env.step(action)
        log_probs.append(log_prob)
        rewards0.append(reward)
        state = torch.Tensor([next_state])

        if done:
            break

    agent.update_parameters(rewards0, log_probs, args2.gamma)


    if i_episode%args2.ckpt_freq == 0:
        torch.save(agent.model.state_dict(), os.path.join(dir, 'reinforce-'+str(i_episode)+'.pkl'))
    if i_episode%1000==0:
        np.save('reward_gumbel_top_K_reinforce_mixorder100.npy',rewards)
        np.save('constraint_gumbel_top_K_reinforce_mixorder100.npy',constraints)

2 2087.716132139086 0.019178082191780823


<ipython-input-186-ff246c2564bb>:80: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  utils.clip_grad_norm(self.model.parameters(), 40)


3 2054.8688048729246 0.010958904109589041
4 2039.171658315331 0.021917808219178082
5 2144.878079334803 0.00821917808219178
6 2015.6511506412548 0.009589041095890411
7 2016.10997360043 0.021917808219178082
8 2018.0345125321737 0.019178082191780823
9 2041.7502844991504 0.021917808219178082
10 2108.6943769838153 0.023287671232876714
11 2060.6796443072185 0.03424657534246575
12 2100.3339864736245 0.06575342465753424
13 1999.4782572372815 0.036986301369863014
14 1998.9084733490974 0.028767123287671233
15 2085.18662284784 0.021917808219178082
16 2095.563740488 0.015068493150684932
17 2053.9375234077315 0.021917808219178082
18 2036.522165061206 0.024657534246575342
19 1996.2458684846365 0.010958904109589041
20 1962.7342099998216 0.01780821917808219
21 2026.8292905921285 0.024657534246575342
22 1963.0982659402466 0.0136986301369863
23 2040.9200166687865 0.02054794520547945
24 2106.9488386726225 0.026027397260273973
25 2001.7477317199514 0.00684931506849315
26 2121.274928393963 0.01095890410958

196 2043.1359391851436 0.00821917808219178
197 2005.298195642294 0.015068493150684932
198 2083.9694689238886 0.010958904109589041
199 2104.4950961016616 0.010958904109589041
200 1979.379737571048 0.0136986301369863
201 1979.38860898162 0.010958904109589041
202 1976.3420258037959 0.00684931506849315
203 2078.696404824754 0.026027397260273973
204 2019.2102453365642 0.005479452054794521
205 2078.7939771832566 0.010958904109589041
206 1967.4278985417122 0.019178082191780823
207 2020.8247001824986 0.01780821917808219
208 2025.4036909798322 0.01643835616438356
209 1996.1699862207092 0.028767123287671233
210 1988.7668112033725 0.015068493150684932
211 2034.1827562692129 0.04246575342465753
212 2093.1495832102282 0.038356164383561646
213 1976.9259381764286 0.021917808219178082
214 2058.065301958737 0.03424657534246575
215 2061.7118698171503 0.03561643835616438
216 2099.354447064807 0.010958904109589041
217 1927.507224789028 0.010958904109589041
218 2000.289813987532 0.012328767123287671
219 20

386 2047.5231073295427 0.026027397260273973
387 2134.295214894685 0.023287671232876714
388 1971.8133273681042 0.023287671232876714
389 2101.7261404450987 0.019178082191780823
390 2038.2584340726198 0.010958904109589041
391 1972.5810450495946 0.009589041095890411
392 2047.2031429481547 0.026027397260273973
393 1985.124958385849 0.021917808219178082
394 1932.529613170733 0.012328767123287671
395 2021.2063061309002 0.015068493150684932
396 2077.969022461308 0.00821917808219178
397 1980.2718000508858 0.028767123287671233
398 2099.907006002058 0.038356164383561646
399 1959.3493154646503 0.02054794520547945
400 2061.761958030891 0.005479452054794521
401 1966.9735991577745 0.00410958904109589
402 1997.3589585663647 0.03972602739726028
403 2050.519342829155 0.01780821917808219
404 1965.6032950935216 0.026027397260273973
405 2012.337102286097 0.03287671232876712
406 2081.3360143013833 0.023287671232876714
407 2041.7205187090312 0.019178082191780823
408 2117.112661260466 0.01780821917808219
409 

...

4126 1988.2345676246255 0.0027397260273972603
4127 2046.3701300029968 0.026027397260273973
4128 2024.5105370880046 0.009589041095890411
4129 2078.0565799832275 0.00684931506849315
4130 2076.5358829829584 0.015068493150684932
4131 1936.1152487927475 0.02054794520547945
4132 2087.3204523702957 0.031506849315068496
4133 2096.6148206562357 0.00821917808219178
4134 2043.2092961261262 0.0547945205479452
4135 2041.4005571367884 0.01780821917808219
4136 1969.9110471717975 0.00821917808219178
4137 1997.9598043383635 0.0136986301369863
4138 2049.4842450315227 0.030136986301369864
4139 2058.030133907251 0.024657534246575342
4140 2016.438578529891 0.012328767123287671
4141 2058.582115142648 0.030136986301369864
4142 2063.50037964092 0.03561643835616438
4143 2008.895003030964 0.026027397260273973
4144 1994.0518728657514 0.024657534246575342
4145 2054.89549149321 0.01780821917808219
4146 2040.5371755101444 0.024657534246575342
4147 2029.7705753661858 0.036986301369863014
4148 2108.037293965953 0.013

4312 2058.271946509541 0.015068493150684932
4313 2076.9270815452846 0.01643835616438356
4314 2076.3617851697218 0.00821917808219178
4315 2077.5799854136417 0.021917808219178082
4316 1992.582975905382 0.009589041095890411
4317 1999.3533204676294 0.0136986301369863
4318 2087.627837403606 0.030136986301369864
4319 2077.0399387417033 0.01643835616438356
4320 2054.4043428242476 0.024657534246575342
4321 2100.5766228645634 0.0136986301369863
4322 2017.3871399367513 0.023287671232876714
4323 1922.3297442998503 0.024657534246575342
4324 2051.0362668799303 0.0273972602739726
4325 2064.3983157214466 0.024657534246575342
4326 2117.0173587698523 0.028767123287671233
4327 1954.3851658775384 0.03287671232876712
4328 2131.5809337587593 0.012328767123287671
4329 2031.163344525723 0.00821917808219178
4330 1943.6945907801805 0.030136986301369864
4331 2051.6860249103024 0.019178082191780823
4332 2059.3003207812453 0.02054794520547945
4333 2018.8860906483799 0.0273972602739726
4334 2075.5613644515242 0.01

4498 2059.8450886705523 0.00821917808219178
4499 2038.4896462141255 0.043835616438356165
4500 2037.5321943535996 0.021917808219178082
4501 2047.7236627510097 0.015068493150684932
4502 2065.0243643018566 0.010958904109589041
4503 2161.351399015186 0.0273972602739726
4504 2093.2911774672184 0.01780821917808219
4505 2016.8015918222295 0.015068493150684932
4506 1942.9256754414528 0.024657534246575342
4507 2025.1705526799608 0.023287671232876714
4508 2057.902910276071 0.00821917808219178
4509 2087.3767320004017 0.04794520547945205
4510 2001.6071111874385 0.009589041095890411
4511 1990.8465675785012 0.024657534246575342
4512 2098.441623076742 0.01780821917808219
4513 2060.439130682609 0.03287671232876712
4514 2049.988050376797 0.010958904109589041
4515 1964.109017875373 0.021917808219178082
4516 2088.500579590204 0.0273972602739726
4517 2034.705735070794 0.03287671232876712
4518 2133.7359206857745 0.01643835616438356
4519 2073.761104080338 0.030136986301369864
4520 1999.2642424934386 0.02739

4684 1928.7322938608013 0.00684931506849315
4685 2038.584978153975 0.015068493150684932
4686 2013.0136791382836 0.010958904109589041
4687 2039.1319158222996 0.009589041095890411
4688 2067.308375914711 0.031506849315068496
4689 2041.712294214651 0.02054794520547945
4690 2108.6934220794374 0.01643835616438356
4691 2047.3571161508257 0.012328767123287671
4692 2037.7322454019218 0.015068493150684932
4693 2078.4077967502876 0.030136986301369864
4694 2066.664697915875 0.015068493150684932
4695 2007.4403131845447 0.019178082191780823
4696 2049.0345095563093 0.023287671232876714
4697 1997.733851791528 0.015068493150684932
4698 2089.2631477230166 0.036986301369863014
4699 2018.7726186826287 0.036986301369863014
4700 1980.182437302287 0.03972602739726028
4701 2056.4620962867925 0.0273972602739726
4702 2086.125884316182 0.009589041095890411
4703 2099.1359353301423 0.012328767123287671
4704 1986.812473927835 0.0136986301369863
4705 2011.8697847747726 0.024657534246575342
4706 2077.4933976934362 0.

4870 2069.2921604883177 0.021917808219178082
4871 2051.5602189664964 0.04246575342465753
4872 2037.7375756524727 0.026027397260273973
4873 2031.6310470435847 0.00684931506849315
4874 1969.9755356646906 0.0136986301369863
4875 2079.385604671274 0.0410958904109589
4876 2078.0231906038407 0.00821917808219178
4877 2038.408440373572 0.019178082191780823
4878 2010.6826835421828 0.012328767123287671
4879 2139.395024766742 0.00821917808219178
4880 1975.414354439315 0.02054794520547945
4881 2097.2794230387362 0.01643835616438356
4882 2086.0308187600385 0.010958904109589041
4883 2045.4032429649126 0.00410958904109589
4884 2069.887735369911 0.031506849315068496
4885 2045.1834258764236 0.0136986301369863
4886 1928.751864660068 0.023287671232876714
4887 2037.695404344885 0.01643835616438356
4888 2077.8630822450205 0.010958904109589041
4889 2084.778222745178 0.03561643835616438
4890 2025.9260748339445 0.0136986301369863
4891 1940.3670611747477 0.028767123287671233
4892 2075.2563547908744 0.016438356

6

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.])